# Astro 528, Lab 7, Exercise 2
## GPU Computing II:  Broadcasting, Fusion, Reductions

In this lab exercise, we'll build on programming pattern for harnessing GPUs with a very high-level interface. This assumes that you've successfully gotten GPUs working with Julia from [exercise 1](ex1.ipynb).  Again, students are advised to run the exercises in this lab on ICS-ACI rather than their own system.  

The ICS-ACI Jupyter notebook server and interactive desktop provided by the ICS-ACI portal are now using interactive notes that include a GPU, and students should be able to run all the code there.  However, the GPUs on the interactive nodes are relatively modest GPUs.  You may want to go ahead and submit ex2.pbs, so that it can queue and run on a CyberLAMP GPU node while you work through this notebook.  Then you can review the output to compare the results and performance of the GPUs on the interactive node to that of the CyberLAMP GPU nodes.  For submitting jobs, you'll use the [command line interface](https://ics.psu.edu/computing-services/ics-aci-user-guide/#05-00-basics-aci-resources) to submit the PBS jobs, following a similar syntax as the [lab's 6 README](https://github.com/PsuAstro528/lab6-start/blob/master/README.md).  

## Setting up
First, we'll make sure that `CUDAHOME` is set, activate the project in the current directory and load packages to be used. Here we assume that you've already instantiated this project during exercise 1.

In [ ]:
if haskey(ENV,"CUDAHOME")
    println("The CUDAHOME environment variable has been set to: ",ENV["CUDAHOME"])
else
    println("The CUDAHOME environment variable has not been set.  Setting it now...")
    ENV["CUDAHOME"] = "/gpfs/group/ebf11/default/astro528/cuda"
end

In [ ]:
using Pkg
Pkg.activate(".")

In [ ]:
#using CUDAdrv

This provides very basic functionality, like querying what version of the CUDA Toolkit is being used,

In [ ]:
using CUDAnative, CuArrays, GPUArrays

In [ ]:
using BenchmarkTools, Statistics

In [ ]:
using Plots

In [ ]:
#pyplot()  # in case gr() gives you trouble

## Writing GPU-compatiable functions for broadcasting


You can write high-level julia code for GPUs even if they have more than linear algebra.  Here I'll show a demonstration of solving [Kepler's equation](https://en.wikipedia.org/wiki/Kepler%27s_equation) and computing the radial velocity of a star due to its reflex motion due to a planet following a Keplerian orbit.  Kepler's equation describes the motion in time of a body on a Keplerian orbit.  The "mean anomaly" (M) increases linearly with time, while the "eccentric anomaly" (E) gives the angular position of the body.  Kepler's equation is $$M = E - e \sin(E),$$ where $M$ is the mean anomaly, $E$ is the eccentric anomaly, and $e$ is the orbital eccentricity.  While it is trivial to compute the mean anomaly from the eccentric anomaly, computing the eccentric anomaly at a given time requires an itterative solver, which makes this relatively slow (compared to other parts of orbit calculation) and hence a good candidate for implementing on a GPU.  

To keep the notebook relatively clean, the code for solving Kepler's equation is in [src/kepler_eqn.jl](src/kepler_eqn.jl).  First, let's load that code and its module.

In [ ]:
include("src/kepler_eqn.jl")
using .KeplerEqn

#=
"Solve Computing Eccentric Anomaly from Mean Anomally via Kepler's Equation"
module KeplerEqn

export calc_ecc_anom
export calc_ecc_anom_cpu, calc_ecc_anom_gpu
using CUDAnative

"""   ecc_anom_init_guess_danby(M, ecc)
Initial guess for eccentric anomaly given mean anomaly (M) and eccentricity (ecc)
    Based on "The Solution of Kepler's Equations - Part Three"  
    Danby, J. M. A. (1987) Journal: Celestial Mechanics, Volume 40, Issue 3-4, pp. 303-312  1987CeMec..40..303D
"""
function ecc_anom_init_guess_danby(M, ecc)
    k = convert(typeof(ecc),0.85)
    if(M<zero(M)) M += 2pi end
    (M<pi) ? M + k*ecc : M - k*ecc;
end

"""   update_ecc_anom_laguerre(E, M, ecc)
Update the current guess (E) for the solution to Kepler's equation given mean anomaly (M) and eccentricity (ecc)
   Based on "An Improved Algorithm due to Laguerre for the Solution of Kepler's Equation"
   Conway, B. A.  (1986) Celestial Mechanics, Volume 39, Issue 2, pp.199-211  1986CeMec..39..199C
"""
function update_ecc_anom_laguerre(E, M, ecc)
  es = ecc*sin(E)
  ec = ecc*cos(E)
  F = (E-es)-M
  Fp = one(E)-ec
  Fpp = es
  n = 5
  root = sqrt(abs((n-1)*((n-1)*Fp*Fp-n*F*Fpp)))
  denom = Fp>zero(E) ? Fp+root : Fp-root
  return E-n*F/denom
end

"Same as update_ecc_anom_laguerre_cpu, except uses GPU math functions"
function update_ecc_anom_laguerre_gpu(E, M, ecc)
  es = ecc*CUDAnative.sin(E)
  ec = ecc*CUDAnative.cos(E)
  F = (E-es)-M
  Fp = one(E)-ec
  Fpp = es
  n = 5
  root = CUDAnative.sqrt(abs((n-1)*((n-1)*Fp*Fp-n*F*Fpp)))
  denom = Fp>zero(E) ? Fp+root : Fp-root
  return E-n*F/denom
end

"Loop to update the current estimate of the solution to Kepler's equation"
function calc_ecc_anom_itterative_laguerre(mean_anom, ecc, tol, max_its)
    M = mod(mean_anom,convert(typeof(mean_anom),2pi))
    E = ecc_anom_init_guess_danby(M,ecc)
    for i in 1:max_its
       E_old = E
       E = update_ecc_anom_laguerre(E_old, M, ecc)
       if abs(E-E_old)<tol break end
    end
    return E
end

"Loop to update the current estimate of the solution to Kepler's equation.  Calls GPU update"
function calc_ecc_anom_itterative_laguerre_gpu(mean_anom, ecc, tol, max_its)
    @assert zero(ecc) <= ecc < one(ecc)
    @assert tol*100 <= one(tol)
    M = mod(mean_anom,convert(typeof(mean_anom),2pi))
    E = ecc_anom_init_guess_danby(M,ecc)
    for i in 1:max_its
       E_old = E
       E = update_ecc_anom_laguerre_gpu(E_old, M, ecc)
       if abs(E-E_old)<tol break end
    end
    return E
end

const default_max_its_laguerre = 200
const default_ecc_anom_tol_cpu = 1e-8
const default_ecc_anom_tol_gpu = 1f-8
"Calculate eccentric anomaly given mean anomaly and eccentricty (in radians)"
calc_ecc_anom_cpu(mean_anom, ecc) = calc_ecc_anom_itterative_laguerre(mean_anom, ecc, default_ecc_anom_tol_cpu, default_max_its_laguerre)
calc_ecc_anom_gpu(mean_anom, ecc) = calc_ecc_anom_itterative_laguerre_gpu(mean_anom, ecc, default_ecc_anom_tol_gpu, default_max_its_laguerre)
calc_ecc_anom(mean_anom, ecc) = calc_ecc_anom_cpu(mean_anom, ecc)

end # module KeplerEqn
=#

Next, we'll create a simple 2-d grid of values of the eccentricity and mean anomaly.  (Feel free to remove the `;`'s to inspect the results.)

In [ ]:
num_ecc = 100
num_mean_anom = 200
max_ecc = 0.9
ecc_h = [max_ecc*i/num_ecc for i in 0:(num_ecc-1), j in 0:(num_mean_anom-1) ]
mean_anom_h = [2pi*j/num_mean_anom for i in 0:(num_ecc-1), j in 0:(num_mean_anom-1) ];

As before, we can put these into GPUArrays and perform the calculation on the GPU using Julia's dot-syntax for broadcasting the function to each element of the arrays.

In [ ]:
ecc_d = cu(ecc_h)
mean_anom_d = cu(mean_anom_h)
ecc_anom_d = calc_ecc_anom.(mean_anom_d,ecc_d)

Now, we can compare the time to compute the eccentric anomaly at each point in our grid using the CPU and GPU.

In [ ]:
@btime ecc_anom_h = calc_ecc_anom.(mean_anom_h,ecc_h) samples=5
CuArrays.@time CuArrays.@sync( ecc_anom_d = calc_ecc_anom.(mean_anom_d,ecc_d) )
CuArrays.@time CuArrays.@sync( ecc_anom_d = calc_ecc_anom.(mean_anom_d,ecc_d) )

At this point, the GPU was likely slower than the CPU.  We can further improve the performance by having the GPU write directly to a pre-allocated array, using `.=`.  While this will give us a speed boost, the calculation is performed entirely on the GPU and that comes with some limitations.  In particular, the default `sin` and `sqrt` functions have error handling built in that isn't compatible with the GPU.  The [CUDAnative.jl](https://github.com/JuliaGPU/CUDAnative.jl) package provides access to numerous CUDA-specific function that we can use to work around this problem.  We'll simply  replace calls to `sin(x)` with calls to `CUDAnative.sin(x)` (and similar for `sqrt`).  That's been implemented that in a similar function calc_ecc_anom_gpu, so you can see how to use this technique for your project.

In [ ]:
ecc_anom_d = cuzeros(size(mean_anom_d))
CuArrays.@time CuArrays.@sync( ecc_anom_d .= calc_ecc_anom_gpu.(mean_anom_d,ecc_d) )
CuArrays.@time CuArrays.@sync( ecc_anom_d .= calc_ecc_anom_gpu.(mean_anom_d,ecc_d) )

For fun, we can plot our results for the eccentric anomaly as a function of mean anomaly and eccentricity.

In [ ]:
ecc_anom_d .= calc_ecc_anom_gpu.(mean_anom_d,ecc_d)
GPUArrays.synchronize(ecc_anom_d)

In [ ]:
#using Plots   # In case you didn't load Plots earlier
#pyplot()

In [ ]:
plot(Array(mean_anom_d[1,:]), Array(ecc_d[:,1]), ecc_anom_d, st = :contourf)

## Minimizing launch overhead in GPU calculations: Kernel fusion
Of course, we usually want to perform more complicated calculations than just solving one equation.  For example, we might want to solve Kepler's equation so that we can compute the radial velocity of a star under a given model and compare that to observations.  Think back to the time required to launch a GPU kernel compared to the time to perform it's work.  If we launch lots of small kernels and/or constantly move data back and forth between the GPU and CPU memory systems, we won't be able to realize the computational benefits of a GPU.  Therefore, we'll want to "fuse" multiple operations into one kernel and keep most results on the GPU and only move back the final results we need. The following example demonstrates this programming pattern.

While I put the code for solving Kepler's equation in a separate file, now I'd like you to pay closer attention to syntax of writing functions for a GPU, so I've left the code for computing the radial velocity due to a planet on a Keplerian orbit in the notebook.

In [ ]:
"""   calc_rv_one_planet(P,K,h,k,M0, time) 
Calculate radial velocity of star due to planet following Keplerian orbit at specified time.
Inputs:
- Period
- RV amplitude
- h = e sin(omega)
- k = e cos(omega)
- mean anomaly at time=0
- time
This differs from usual expressions so as to be differntiable, even at zero eccentricity.  
Based on "An analytical solution for Kepler's problem"
Pál, András, Monthly Notices of the Royal Astronomical Society, 396, 3, 1737-1742.  2009MNRAS.396.1737P
"""
function calc_rv_one_planet(P,K,h,k,M0, time)
  ecc = sqrt(h*h+k*k)
  T = typeof(P)
  w = atan(k,h)
  n = T(2pi)/P
  M = time*n-M0
  E = calc_ecc_anom(M,ecc)
  lambda = w+M
  lpesE = lambda+ecc*sin(E)  
  c = cos(lpesE)
  s = sin(lpesE)
  @assert(0.0<=ecc<1.0)
  j = sqrt((one(T)-ecc)*(one(T)+ecc))
  p, q = (ecc == zero(T)) ? (zero(T), zero(T)) : (ecc*sin(E), ecc*cos(E))
  a = K/(n/sqrt((one(T)-ecc)*(one(T)+ecc)))
  zdot = a*n/(one(T)-q)*( cos(lambda+p)-k*q/(one(T)+j) )
end

function calc_rv_one_planet_gpu(P,K,h,k,M0, time)
  ecc = CUDAnative.sqrt(h*h+k*k)
  T = typeof(P)
  w = CUDAnative.atan2(k,h)
  n = T(2pi)/P
  M = mod(time*n-M0,T(2pi))
  E = calc_ecc_anom_gpu(M,ecc)
  lambda = w+M   #lambda = mod(w+M,T(2pi))
  lpesE = lambda+ecc*CUDAnative.sin(E)
  c = CUDAnative.cos(lpesE)
  s = CUDAnative.sin(lpesE)
  @assert(0.0<=ecc<1.0)
  j = CUDAnative.sqrt((one(T)-ecc)*(one(T)+ecc))
  p, q = (ecc == zero(T)) ? (zero(T), zero(T)) : (ecc*CUDAnative.sin(E), ecc*CUDAnative.cos(E))
  a = K/(n/CUDAnative.sqrt((one(T)-ecc)*(one(T)+ecc)))
  zdot = a*n/(one(T)-q)*( CUDAnative.cos(lambda+p)-k*q/(one(T)+j) )
end

First, notice that each function takes a list of scalars.  We'll use Julia's dot broadcasting to operate on arrays.  Previously, we discussed the danger of having a function take a long list of arguments.  Consider the functions above a low-level interface that a GPU programmer would write and test once, but users wouldn't use.  Then we can reduce the risk of problems and make the code more convenient by defining a composite structure to contain the planet's RV parameters, along with helper functions that unpack the composite type and call the low-level function.

In [ ]:
struct RvPlanetParam{T<:Number}
   P::T
   K::T
   h::T
   k::T
   M0::T
end

function calc_rv_one_planet(theta::RvPlanetParam, time)
  P,K,h,k,M0 = theta.P, theta.K, theta.h, theta.k, theta.M0
  calc_rv_one_planet(P,K,h,k,M0, time)
end

function calc_rv_one_planet_gpu(theta::RvPlanetParam, time)
  P,K,h,k,M0 = theta.P, theta.K, theta.h, theta.k, theta.M0
  calc_rv_one_planet_gpu(P,K,h,k,M0, time)
end

Now, we could create a nice interface for setting and accessing the data in `RvPlanetParms`.  At the same time, this pattern allows both the CPU and the GPU to store and access to the data in a very efficient way.  In particular, Julia's dot broadcasting allows us to pass separate arrays for each parameter.  For example,

In [ ]:
RvPlanetParam.([4.0,5.0],[10.0,9.0],[0.05,0.05],[-0.1,0.1],[0.1*pi,0.2*pi])

Now, let's create a set of simulated observations times over ~8 months.  This time, instead of generating the data on the CPU and moving it to the GPU, we'll generate the random values on the GPU (using the `CuArrays` interface to the [CURAND](https://docs.nvidia.com/cuda/curand/index.html) library from the CUDA toolkit), perform arithmetic on the GPU, and then sort the resulting times, all leaving the data on the GPU.

In [ ]:
import CuArrays: CURAND.curand, CURAND.curandn

In [ ]:
num_obs = 200
min_obs_time = 0.0f0
max_obs_time = 240.0f0
times_d = min_obs_time .+ (max_obs_time-min_obs_time) .* curand(num_obs) 
sort!(times_d)

And definte the true parameters for a simulated data set.

In [ ]:
P_true = 4.0
K_true = 10.0
h_true = 0.05
k_true = 0.10
M0_true = pi/4
sigma_obs = 1.0f0
RvPlanetParamT = GPUArrays.supports_double(0) ? Float64 : Float32
param_h = [RvPlanetParam{RvPlanetParamT}(P_true,K_true,h_true,k_true,M0_true)]

We can compute the radial velocities under the true model, either on the GPU.

In [ ]:
rvs_true_d = calc_rv_one_planet.(param_h,times_d)
rvs_obs_d = rvs_true_d .+ sigma_obs .* curandn(num_obs)

Note that we passed an array of RvPlanetParam, so we're using good programming practices, and our helper function allows Julia to convert that into call the low-level function for the GPU.  We can bring the results back to the CPU using `Array`.

In [ ]:
rvs_obs_h = Array(rvs_obs_d)

Often, we need to explore a large parameter space to search for a good model.  For sake of an example, we'll use a simple Monte Carlo search.  Of course, it is usually wise to use a more efficient algorithm.

In [ ]:
num_param = 10000
Plist_d = P_true .* ( 1.0 .+ 0.01 .* (curand(num_param).-0.5) )
Klist_d = K_true .* ( 1.0 .+ 0.1  .* (curand(num_param).-0.5) )
hlist_d = h_true .+ 0.1 .* (curand(num_param).-0.5)
klist_d = k_true .+ 0.1 .* (curand(num_param).-0.5)
M0list_d = M0_true .+ 2pi .* 0.1 .* (curand(num_param).-0.5)
CuArrays.@time CuArrays.@sync ( params_d = RvPlanetParam{RvPlanetParamT}.(Plist_d, Klist_d, hlist_d, klist_d, M0list_d); )

Now we'll want to evaluate the Planet's velocity at each time and for each set of model parameters.  We can acheive that using Julia's [`repeat`](https://docs.julialang.org/en/v1/base/arrays/#Base.repeat) and [`reshape`](https://docs.julialang.org/en/v1/base/arrays/#Base.reshape) functions to minimize unnecessary copying of the data, just as we would do the CPU.  It may be useful to look at the size and/or contents of the new CuArrays being being created, to help understand the upcoming function call.  

(For some reason the Weave script gave an error when I left out the semi-colons.  If you remove the semi-colons, then I'd suggest restoring the semi-colons after each of the next two cells before submitting this notebook as a batch job.)

In [ ]:
CuArrays.@time expanded_times_d = reshape(repeat(times_d,num_param),num_obs,num_param);

In [ ]:
CuArrays.@time expanded_params_d = reshape(params_d,1,num_param);

How much GPU memory was required for each of the above operations.  How does that compare to creating new arrays of `num_obs`x`num_param`?
    
INSERT RESPONSE

Now, let's compute the radial velocity at each time for each set of parameters.  We'll time the results and compare to the time on the CPU.  Remember, that the first call to a function with a given set of parameter types results in an expensive compile operation on either the CPU or GPU.  So we'll run each twice.

In [ ]:
rvs_d = cuzeros(num_param,num_obs)
CuArrays.@time CuArrays.@sync( rvs_d .=  calc_rv_one_planet_gpu.(reshape(params_d,num_param,1), reshape(repeat(times_d,num_param),num_obs,num_param)' ) )
CuArrays.@time CuArrays.@sync( rvs_d .=  calc_rv_one_planet_gpu.(reshape(params_d,num_param,1), reshape(repeat(times_d,num_param),num_obs,num_param)' ) )

In [ ]:
params_h = Array(params_d)
times_h = Array(times_d)
@time rvs_h = calc_rv_one_planet.(reshape(params_h,num_param,1), reshape(repeat(times_h,num_param),num_obs,num_param)' )
@time rvs_h = calc_rv_one_planet.(reshape(params_h,num_param,1), reshape(repeat(times_h,num_param),num_obs,num_param)' )

How does the GPU and CPU performance compare for this calculation?
    
INSERT RESPONSE

How large are the differences between the calculations on the CPU and GPU?  
Once you've run this notebook on a dedicated CyberLAMP GPU node, come back and expand on your comparison, noting any difference between the interactive and CyberLAMP nodes.

INSERT RESPONSES

In [ ]:
maximum(abs.(Array(rvs_d).-rvs_h))

## Minimizing communications overhead in GPU calculations: Reductions
So far we've compute the velocity at each time.  Often we'd be most interested in the quality of the fit summing the differences between the model and data at each time.  We can do that, keeping all the data on the GPU using `sum`.  (GPUArrays also provided `prod` for prodcut reductions and a more generic `reduce`.)

In [ ]:
chisq_d = vec(sum(((rvs_d.-rvs_obs_d')./sigma_obs).^2,dims=2) )

By performing the reduction on the GPU, we get faster calculation of $\chi^2$, but more importantly, we reduce the amount of data that would need to be sent from the GPU to the CPU's memory system.

In [ ]:
CuArrays.@time CuArrays.@sync( chi_sq_d = sum(((rvs_d.-rvs_obs_d')./sigma_obs).^2,dims=2) )

In [ ]:
CuArrays.@time CuArrays.@sync( rvs_h = Array(rvs_d) )

In [ ]:
CuArrays.@time CuArrays.@sync( chi_sq_h = Array(chi_sq_d) )

How much longer does it take to send the array of all velocities back to the host, as compared to sending back only the $chi^2$ values?

INSERT RESPONSE

In some cases, we may not even need to send all of the results back to the CPU.  For example, maybe we just want to find the best fitting model

In [ ]:
CuArrays.@time (chisq_good, idx_good) = findmin(chisq_d)
CuArrays.@time good_param = params_d[idx_good]

How does the time required to send back just the parameters from the best-fit model (so far) compare to the time required to send back all of the values of $\chi^2$? 

Just for fun, we can compare those values to the true parameters that we've kept on the host.

In [ ]:
println("True param: ",param_h[1])

In [ ]:
println("Differences: ", map(fn->getfield(good_param,fn)-getfield(param_h[1],fn), fieldnames(typeof(good_param)) ) )
println("ChiSq: ", chisq_good)

And graphically compare the predictions of the true model and the one we found, first in time and then in orbital phase.

In [ ]:
scatter(times_h,calc_rv_one_planet.(param_h,times_h), markersize=2, xlabel="Time (d)", ylabel="RV (m/s)", label="True")
scatter!(times_h,calc_rv_one_planet.([good_param],times_h), label="Model", markersize=2)

In [ ]:
scatter(2pi/param_h[1].P.*mod.(times_h.-param_h[1].M0,param_h[1].P),calc_rv_one_planet.(param_h,times_h), markersize=2, xlabel="Phase", ylabel="RV (m/s)", label="True")
scatter!(2pi/param_h[1].P.*mod.(times_h.-param_h[1].M0,param_h[1].P),calc_rv_one_planet.([good_param],times_h), label="Model", markersize=2)

## Implications for your Class Project
Now that you've seen a tour of how to perform basic GPU computing, do you think that GPU computing would be likely to provide a significant performance benefit for the calculations in your class project?  Why or why not?
        
INSERT RESPONSE